# Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#import twint

I decided to use Twint to scrape data from Twitter. I hesitated with Tweepy but this library is limited with the API limitations... As I want to collect all Twitter mentions from September, that library might not be well-suited.

# Twitter data collection

In [3]:
#configuration
#config = twint.Config()
#config.Search = "PS5"
#config.Lang = "en"
#config.Limit = 5
#config.Since = '2020–09–01 00:01:00'
#config.Until = "2020–9–28 00:00:00"
#config.Store_csv = True
#config.Output = "test.csv"
#running search
#twint.run.Search(config)

In [4]:
#import twitter_scraper

In [5]:
#from twitter_scraper import get_tweets

In [6]:
#tweets = get_tweets("PS5",pages=5)

In [7]:
#tweets

In [8]:
#tweets_df = pd.DataFrame()

In [9]:
#for tweet in tweets:
    #print('Keys:', list(tweet.keys()), '\n')
    #break

In [10]:
#for tweet in tweets:
    #_ = pd.DataFrame({'text' : [tweet['text']],
                    #'isRetweet' : tweet['isRetweet'],
                    #'replies' : tweet['replies'],
                    #'retweets' : tweet['retweets'],
                    #'likes' : tweet['likes']
                    #})
    #tweets_df = tweets_df.append(_, ignore_index = True)
#tweets_df.head()

In [11]:
#for i in tweets:
    #tweets_df.append(i)

In [12]:
#tweets_df

In [13]:
#import twitterscraper
#import datetime as dt

In [14]:
#from twitterscraper import query_tweets

In [15]:
#tweets = query_tweets('bitcoin', limit=10, begindate=dt.date.today(), enddate=dt.date.today(), lang='en')

In [16]:
#for i in tweets:
    #print(i)

In [17]:
#df = pd.DataFrame(t.__dict__ for t in tweets)

In [18]:
#df

In [19]:
#import GetOldTweets3 as got

In [20]:
#tweetCriteria = got.manager.TweetCriteria().setQuerySearch('PS5')\
                                           #.setSince("2020-09-01")\
                                           #.setUntil("2020-09-30")\
                                           #.setMaxTweets(10)
#tweet = got.manager.TweetManager.getTweets(tweetCriteria)[0]
#print(tweet.text)

In [21]:
#import twint
#c = twint.Config()
#c.Search = "ps5"
#c.Store_object = True
#c.Limit = 100
#twint.run.Search(c)
#tlist = c.search_tweet_list

In [22]:
#c=twint.Config()
#c.Hide_output=False #but it ignores this
#c.Store_object=True
#c.Store_csv=True
#c.Since="2020-09-01"
#c.User_full=True
#c.Limit=100 #I have tested up to 1000 ok
#c.Output="testing.csv" #but it ignores this

#srchname="(from:RealDonaldTrump)"
#srch=srchname+" max_id:"+str(1307047063197224961)

#twint.run.Search(c)
#tlist = c.search_tweet_list
#print(tlist)

# YouTube data extraction

At first, I wanted to analyze all PS5 and Xbox tweets in September as many announcements occured that month (price, games, etc.). But it was too difficult for me to get the Twitter data. The API limitations only let me have access to the last 7 days of data. I also had a look at the scrapers (Twint, tweetscraper, GetOldTweets3, etc.) but they are all down for some reasons...
Therefore, I decided to change a bit the scope of the project.

I will have a look at the YouTube comments of both PS5 and Xbox games showcases.
The PS5 showcase occured on the 16th of September while the Xbox Series X showcase occured on the 23rd of July.

In [227]:
Xbox_data = pd.read_fwf('Xbox_comments.txt',header=None)

In [228]:
Xbox_data.fillna('',inplace=True)

In [229]:
#Consolidating text together in one column
Xbox_data['Text'] = Xbox_data[2]
Xbox_data.drop(2,axis=1,inplace=True)
for i in range(3,21):
    Xbox_data['Text']=Xbox_data['Text'] + " " + Xbox_data[i]
    Xbox_data.drop(i,axis=1,inplace=True)

In [230]:
#Removing ":" from end of column "1" (hours)
Xbox_data[1]=Xbox_data[1].apply(lambda x: x[:-1])

In [231]:
#Renaming columns
Xbox_data.rename(columns={0:'Date',1:'Hour'},inplace=True)

In [232]:
Xbox_data.tail(60)

,Date,Hour,Text
1093,2020-08-26,16:01:42,AhahahahahahahahaHhahahaHHHhHHahahahahahahahah...
1094,2020-08-27,21:48:40,#FALLGUYSONXBOX
1095,2020-08-30,09:46:18,RIP
1096,2020-08-30,14:29:26,6:34
1097,2020-08-30,16:26:11,xbox took an L
1098,2020-08-30,23:57:07,Hola
1099,2020-09-01,16:18:27,Thank you for using Persian language ...
1100,2020-09-01,16:18:47,Thank you for using Persian ️️️
1101,2020-09-05,15:13:16,Lol ripbox serious x with failo infuckety ...
1102,2020-09-08,18:32:33,6:25 sounds just like the Cabal in Destiny! ...


# Text processing

In [233]:
import nltk
import re

In [234]:
#We need to install the "stopwords" corpus in order to remove the stopwords later on
#nltk.download()

In [235]:
#Creating a function to clean the comments
def clean_text(text):
    text = re.sub(r'https?:\/\/\S+', '', text) #Removing hyperlinks
    text = re.sub(r"\d{2}:\d{2}s*", '', text) #Removing timestamps XX:XX
    text = re.sub(r"\d{1}:\d{2}s*", '', text) #Removing timestamps X:XX
    text = re.sub('[^A-Za-z0-9 ]+', '', text) #Removing special characters
    return text

In [236]:
Xbox_data['Text']=Xbox_data['Text'].apply(clean_text)

In [237]:
Xbox_data.tail(60)

,Date,Hour,Text
1093,2020-08-26,16:01:42,AhahahahahahahahaHhahahaHHHhHHahahahahahahahah...
1094,2020-08-27,21:48:40,FALLGUYSONXBOX
1095,2020-08-30,09:46:18,RIP
1096,2020-08-30,14:29:26,
1097,2020-08-30,16:26:11,xbox took an L
1098,2020-08-30,23:57:07,Hola
1099,2020-09-01,16:18:27,Thank you for using Persian language ...
1100,2020-09-01,16:18:47,Thank you for using Persian
1101,2020-09-05,15:13:16,Lol ripbox serious x with failo infuckety ...
1102,2020-09-08,18:32:33,sounds just like the Cabal in Destiny ...
